In [7]:
import pandas as pd
import re
from collections import Counter
import numpy as np
from matplotlib import pyplot as plt
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import seaborn as sns

from gensim.models import LdaModel, CoherenceModel
from gensim import corpora
import spacy, logging, warnings

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score,\
                            recall_score, f1_score, jaccard_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

/Users/demid/opt/anaconda3/lib/python3.9/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /Users/demid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/demid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords.append('code')
stopwords.append('new')
stopwords.append('use')
stopwords.append('code')
stopwords.append('pre')
stopwords.append('would')
stopwords.append('get')

In [10]:
def rem_sw(text):
    words = text.lower().split()
    stop_free_words = [word for word in words if word not in stopwords] 
    stop_free_text = " ".join(stop_free_words)
    return stop_free_text

In [8]:
df1 = pd.read_csv('/Users/demid/Downloads/QueryResults(1).csv', sep=',', engine='python', error_bad_lines=False )

/Users/demid/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
tags = list(map(lambda x: set(re.findall("<(.*?)>", x)), df1['Tags']))
tags = sorted(set.union(*tags))
df1['Tags'] = list(map(lambda x: re.findall("<(.*?)>", x), df1['Tags']))

In [19]:
all_tags = []

for i in range(0, 50000):
    for j in range(len(df1['Tags'].iloc[i])):
        all_tags.append(df1['Tags'].iloc[i][j])

In [20]:
CounterVariable = Counter(all_tags)
most_occur = CounterVariable.most_common(100)

In [22]:
centtags = []
for i in range(0, 100):
    centtags.append(most_occur[i][0])

In [23]:
df1['tags_filtered'] = [[t for t in Tags if t in centtags] for Tags in df1['Tags']]

In [25]:
df1["clean_title"] = df1["Title"].apply(lambda s: ' '.join(re.sub("(w+://S+)", " ", s).split()))
df1["clean_title"] = df1["clean_title"].apply(lambda s: ' '.join(re.sub("[.,!?):;(-='...@#_>]", " ", s).split()))
df1["clean_title"] = df1["clean_title"].apply(lambda s: ' '.join(re.sub(r'[0-9]+', '', s).split()))
df1['clean_title'] = df1['clean_title'].apply(lambda  s: rem_sw(s))
tokenizer = nltk.RegexpTokenizer(r'\w+')
df1['clean_title'] = df1['clean_title'].apply(lambda  x:  tokenizer.tokenize(x))  
lemmatizer = WordNetLemmatizer()
df1["clean_title"] = df1["clean_title"].apply(lambda tokens: [lemmatizer.lemmatize(token, pos='v') for token in tokens])

In [26]:
df1["clean_body"] = df1["Body"].apply(lambda s: ' '.join(re.sub("<p>", " ", s).split()))
df1["clean_body"] = df1["clean_body"].apply(lambda s: ' '.join(re.sub("(w+://S+)", " ", s).split()))
df1["clean_body"] = df1["clean_body"].apply(lambda s: ' '.join(re.sub("[.,!?):;(-='...@#_>]", " ", s).split()))
df1["clean_body"] = df1["clean_body"].apply(lambda s: ' '.join(re.sub(r'[0-9]+', '', s).split()))
df1['clean_body'] = df1['clean_body'].apply(lambda  s: rem_sw(s))
df1['clean_body'] = df1['clean_body'].apply(lambda  x:  tokenizer.tokenize(x))       
df1["clean_body"] = df1["clean_body"].apply(lambda tokens: [lemmatizer.lemmatize(token, pos='v') for token in tokens])

In [27]:
df2 = df1.drop(df1[['Title', 'Body', 'Tags', 'Id', 'Score', 'ViewCount', 'FavoriteCount',
       'AnswerCount']], axis=1)
df2['text'] = df2['clean_title']   + df2['clean_body']
df3 = df2.drop(df2[['clean_title', 'clean_body']], axis=1)

In [28]:
df3['text']=[" ".join(text) for text in df3['text'].values]

In [29]:
df3.head()

,tags_filtered,text
0,"[java, json, spring, spring-mvc]",spring json request body map java pojo use spr...
1,[android],communicate activity service localservice andr...
2,[shell],set email address mailx command work kornshell...
3,[visual-studio],ankhsvn versus visualsvn currently ankhsvn int...
4,"[c, performance, optimization]",performance difference c performance differenc...


In [93]:
df3.shape

(50000, 2)

In [30]:
mlb = MultiLabelBinarizer()
tag_mlb = mlb.fit_transform(df3['tags_filtered'])

In [31]:
classifier = MultiOutputClassifier(RandomForestClassifier())

clf = Pipeline([('classify', classifier)])

print (clf)

Pipeline(steps=[('classify',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])


In [32]:
X_train_0, X_test_0, y_train, y_test = train_test_split(
    df3['text'], tag_mlb, test_size=0.2, random_state=11)
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train_0)
X_test = vectorizer.transform(X_test_0)

In [94]:
X_train_0.shape

(40000,)

In [35]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
y_predict = dt.predict(X_test)
print("Accuracy : ", accuracy_score(y_test, y_predict))
print("Precision : ", precision_score(y_test, y_predict, average='samples'))
print("Recall : ", recall_score(y_test, y_predict, average='samples'))
print("F1 Score : ", f1_score(y_test, y_predict, average='samples'))
print("Jaccard :", jaccard_score(y_test, y_predict, average='samples'))


Accuracy :  0.164
Precision :  0.5056733333333333
Recall :  0.48128999999999994


/Users/demid/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/demid/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/demid/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


F1 Score :  0.4611915873015873
Jaccard : 0.37978345238095235


/Users/demid/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
classifier1 = MultiOutputClassifier(DecisionTreeClassifier())

clf1 = Pipeline([('classify', classifier1)])

print (clf1)

Pipeline(steps=[('classify',
                 MultiOutputClassifier(estimator=DecisionTreeClassifier()))])


In [ ]:
clf1.fit(X_train, y_train)
print(clf1.score(X_train, y_train))

In [ ]:
y_hat1 = clf1.predict(X_test)

In [43]:
 pip install fastapi 


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 kB 2.8 MB/s eta 0:00:00
  Attempting uninstall: anyio
    Found existing installation: anyio 2.2.0
    Uninstalling anyio-2.2.0:
      Successfully uninstalled anyio-2.2.0
Note: you may need to restart the kernel to use updated packages.


### Word2Vec

In [63]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
import gensim

In [45]:
from gensim.models import Word2Vec as w2v

In [46]:
w = w2v(
    df3['text'],
    min_count=3,  
    sg = 1,       
    window=7      
) 

In [52]:
sentences = df3['text'].to_list()
sentences = [gensim.utils.simple_preprocess(text) for text in sentences]

In [55]:
w2v_size=300
w2v_window=5
w2v_min_count=1
w2v_epochs=100
maxlen = 24 # adapt to length of sentences

In [56]:
w2v_model = gensim.models.Word2Vec(min_count=w2v_min_count, window=w2v_window,
                                                vector_size=w2v_size,
                                                seed=42,
                                                workers=1)

In [57]:
w2v_model.build_vocab(sentences)

In [58]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_epochs)

(634083528, 681145400)

In [59]:
model_vectors = w2v_model.wv

In [60]:
w2v_words = model_vectors.index_to_key

In [61]:
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")

Vocabulary size: 139214
Word2Vec trained


In [64]:
print("Fit Tokenizer ...")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

Fit Tokenizer ...


In [65]:
x_sentences = pad_sequences(tokenizer.texts_to_sequences(sentences),
                                                     maxlen=maxlen,
                                                     padding='post') 

In [66]:
num_words = len(tokenizer.word_index) + 1
print("Number of unique words: %i" % num_words)

Number of unique words: 139215


### Création de la matrice d'embedding¶

In [67]:
print("Create Embedding matrix ...")
w2v_size = 300
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, w2v_size))
i=0
j=0

Create Embedding matrix ...


In [68]:
for word, idx in word_index.items():
    i +=1
    if word in w2v_words:
        j +=1
        embedding_vector = model_vectors[word]
        if embedding_vector is not None:
            embedding_matrix[idx] = model_vectors[word]

In [69]:
word_rate = np.round(j/i,4)
print("Word embedding rate : ", word_rate)
print("Embedding matrix: %s" % str(embedding_matrix.shape))

Word embedding rate :  1.0
Embedding matrix: (139215, 300)


### Création du modèle d'embedding

In [70]:
# Création du modèle

input=Input(shape=(len(x_sentences),maxlen),dtype='float64')
word_input=Input(shape=(maxlen,),dtype='float64')  
word_embedding=Embedding(input_dim=vocab_size,
                         output_dim=w2v_size,
                         weights = [embedding_matrix],
                         input_length=maxlen)(word_input)
word_vec=GlobalAveragePooling1D()(word_embedding)  
embed_model = Model([word_input],word_vec)

embed_model.summary()

2022-06-16 08:37:06.676980: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 24)]              0         
                                                                 
 embedding (Embedding)       (None, 24, 300)           41764500  
                                                                 
 global_average_pooling1d (G  (None, 300)              0         
 lobalAveragePooling1D)                                          
                                                                 
Total params: 41,764,500
Trainable params: 41,764,500
Non-trainable params: 0
_________________________________________________________________


## Exécution du modèle

In [71]:
embeddings = embed_model.predict(x_sentences)
embeddings.shape

1563/1563 [==============================] - 1s 646us/step


(50000, 300)

### Use of the Embedding matrix for Classifiers

    Split the data into text (X) and labels (Y)
    Preprocess X
    Create a word embedding matrix from X
    Create a tensor input from X
    Train a deep learning model using the tensor inputs and labels (Y)
    Make predictions on new data

In [92]:
df3.shape

(50000, 2)

In [76]:
# Split the data into text (X) and labels (Y)
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    df3['text'], tag_mlb, test_size=0.2, random_state=11)

In [91]:
X_train1.shape

(40000,)

In [77]:
# Preprocess X_train1

#sentences = X_train1.to_list()
sentences = [gensim.utils.simple_preprocess(text) for text in X_train1]

In [78]:
w2v_model.build_vocab(sentences)

In [79]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=w2v_epochs)

(505743991, 543106600)

In [80]:
model_vectors = w2v_model.wv

In [81]:
w2v_words = model_vectors.index_to_key

In [82]:
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")

Vocabulary size: 120038
Word2Vec trained


In [83]:
print("Fit Tokenizer ...")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

Fit Tokenizer ...


In [84]:
x_sentences = pad_sequences(tokenizer.texts_to_sequences(sentences),
                                                     maxlen=maxlen,
                                                     padding='post') 

In [85]:
num_words = len(tokenizer.word_index) + 1
print("Number of unique words: %i" % num_words)

Number of unique words: 120039


In [86]:
print("Create Embedding matrix ...")
w2v_size = 300
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, w2v_size))
i=0
j=0

Create Embedding matrix ...


In [87]:
for word, idx in word_index.items():
    i +=1
    if word in w2v_words:
        j +=1
        embedding_vector = model_vectors[word]
        if embedding_vector is not None:
            embedding_matrix[idx] = model_vectors[word]

In [88]:
word_rate = np.round(j/i,4)
print("Word embedding rate : ", word_rate)
print("Embedding matrix: %s" % str(embedding_matrix.shape))

Word embedding rate :  1.0
Embedding matrix: (120039, 300)


In [89]:
# Création du modèle d'embedding

input=Input(shape=(len(x_sentences),maxlen),dtype='float64')
word_input=Input(shape=(maxlen,),dtype='float64')  
word_embedding=Embedding(input_dim=vocab_size,
                         output_dim=w2v_size,
                         weights = [embedding_matrix],
                         input_length=maxlen)(word_input)
word_vec=GlobalAveragePooling1D()(word_embedding)  
embed_model = Model([word_input],word_vec)

embed_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 24)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 24, 300)           36011700  
                                                                 
 global_average_pooling1d_1   (None, 300)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
Total params: 36,011,700
Trainable params: 36,011,700
Non-trainable params: 0
_________________________________________________________________


In [90]:
embeddings1 = embed_model.predict(x_sentences)
embeddings1.shape

1250/1250 [==============================] - 1s 655us/step


(40000, 300)

#### I do the same for X_test

In [97]:
sentences2 = [gensim.utils.simple_preprocess(text) for text in X_test1]

In [98]:
w2v_model.build_vocab(sentences2)

In [99]:
w2v_model.train(sentences2, total_examples=w2v_model.corpus_count, epochs=w2v_epochs)

(128319014, 138038800)

In [101]:
model_vectors = w2v_model.wv

In [102]:
w2v_words = model_vectors.index_to_key

In [103]:
print("Vocabulary size: %i" % len(w2v_words))
print("Word2Vec trained")

Vocabulary size: 47528
Word2Vec trained


In [104]:
print("Fit Tokenizer ...")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences2)

Fit Tokenizer ...


In [105]:
x_sentences2 = pad_sequences(tokenizer.texts_to_sequences(sentences2),
                                                     maxlen=maxlen,
                                                     padding='post') 

In [106]:
num_words = len(tokenizer.word_index) + 1
print("Number of unique words: %i" % num_words)

Number of unique words: 47529


In [107]:
print("Create Embedding matrix ...")
w2v_size = 300
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
embedding_matrix = np.zeros((vocab_size, w2v_size))
i=0
j=0

Create Embedding matrix ...


In [108]:
for word, idx in word_index.items():
    i +=1
    if word in w2v_words:
        j +=1
        embedding_vector = model_vectors[word]
        if embedding_vector is not None:
            embedding_matrix[idx] = model_vectors[word]

In [109]:
word_rate = np.round(j/i,4)
print("Word embedding rate : ", word_rate)
print("Embedding matrix: %s" % str(embedding_matrix.shape))

Word embedding rate :  1.0
Embedding matrix: (47529, 300)


In [110]:
embeddings2 = embed_model.predict(x_sentences2)
embeddings2.shape

313/313 [==============================] - 0s 1ms/step


(10000, 300)

In [96]:
dtW = DecisionTreeClassifier()
dtW.fit(embeddings1, y_train)


DecisionTreeClassifier()

In [113]:
y_predict1 = dtW.predict(embeddings2)
print("Accuracy : ", accuracy_score(y_test1, y_predict1))
print("Precision : ", precision_score(y_test1, y_predict1, average='samples'))
print("Recall : ", recall_score(y_test1, y_predict1, average='samples'))
print("F1 Score : ", f1_score(y_test1, y_predict1, average='samples'))
print("Jaccard :", jaccard_score(y_test1, y_predict1, average='samples'))

Accuracy :  0.0109
Precision :  0.04906
Recall :  0.046518333333333335


/Users/demid/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/demid/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/demid/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


F1 Score :  0.04381912698412698
Jaccard : 0.03210797619047618


/Users/demid/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Jaccard is ill-defined and being set to 0.0 in samples with no true or predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### BERT

In [119]:
import tensorflow as tf
# import tensorflow_hub as hub
import tensorflow.keras
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

# Bert
import os
import transformers
from transformers import *

os.environ["TF_KERAS"]='1'

In [120]:
#pip install sacremoses

In [121]:
#pip install transformers

In [122]:
print(tf.__version__)
print(tensorflow.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.test.is_built_with_cuda())

2.9.1
2.9.1
Num GPUs Available:  0
False


In [123]:
# Fonction de préparation des sentences
def bert_inp_fct(sentences, bert_tokenizer, max_length) :
    input_ids=[]
    token_type_ids = []
    attention_mask=[]
    bert_inp_tot = []

    for sent in sentences:
        bert_inp = bert_tokenizer.encode_plus(sent,
                                              add_special_tokens = True,
                                              max_length = max_length,
                                              padding='max_length',
                                              return_attention_mask = True, 
                                              return_token_type_ids=True,
                                              truncation=True,
                                              return_tensors="tf")
    
        input_ids.append(bert_inp['input_ids'][0])
        token_type_ids.append(bert_inp['token_type_ids'][0])
        attention_mask.append(bert_inp['attention_mask'][0])
        bert_inp_tot.append((bert_inp['input_ids'][0], 
                             bert_inp['token_type_ids'][0], 
                             bert_inp['attention_mask'][0]))

    input_ids = np.asarray(input_ids)
    token_type_ids = np.asarray(token_type_ids)
    attention_mask = np.array(attention_mask)
    
    return input_ids, token_type_ids, attention_mask, bert_inp_tot
    

# Fonction de création des features
def feature_BERT_fct(model, model_type, sentences, max_length, b_size, mode='HF') :
    batch_size = b_size
    batch_size_pred = b_size
    bert_tokenizer = AutoTokenizer.from_pretrained(model_type)
    time1 = time.time()

    for step in range(len(sentences)//batch_size) :
        idx = step*batch_size
        input_ids, token_type_ids, attention_mask, bert_inp_tot = bert_inp_fct(sentences[idx:idx+batch_size], 
                                                                      bert_tokenizer, max_length)
        
        if mode=='HF' :    # Bert HuggingFace
            outputs = model.predict([input_ids, attention_mask, token_type_ids], batch_size=batch_size_pred)
            last_hidden_states = outputs.last_hidden_state

        if mode=='TFhub' : # Bert Tensorflow Hub
            text_preprocessed = {"input_word_ids" : input_ids, 
                                 "input_mask" : attention_mask, 
                                 "input_type_ids" : token_type_ids}
            outputs = model(text_preprocessed)
            last_hidden_states = outputs['sequence_output']
             
        if step ==0 :
            last_hidden_states_tot = last_hidden_states
            last_hidden_states_tot_0 = last_hidden_states
        else :
            last_hidden_states_tot = np.concatenate((last_hidden_states_tot,last_hidden_states))
    
    features_bert = np.array(last_hidden_states_tot).mean(axis=1)
    
    time2 = np.round(time.time() - time1,0)
    print("temps traitement : ", time2)
     
    return features_bert, last_hidden_states_tot